# Import related functions

In [1]:
import sys
sys.path.append(r'./script/')
from ECMpy_function import *

# Input and output files

In [2]:
json_output_file = './model/ecMTM.json'
json_model_file=json_output_file
reaction_kcat_MW = pd.read_csv('./get_reaction_kcat/kcat/reaction_change_by_C13_TurNup24_1.csv',index_col=0)
json_model_file=json_output_file

# Enzyme-constrained model predicts target genes

### Biomass yield target of 10%

In [3]:

name_lst=['mal__L','fum','etoh']
for name in name_lst:
    enz_model=get_enzyme_constraint_model(json_model_file)
    product = enz_model.metabolites.get_by_id(name+'_e')
    demand = enz_model.add_boundary(product, type='demand')  # add demand reaction as the objective 
    enz_model.objective =demand
    substrate='R2399_reverse'
    enz_model.reactions.get_by_id("R1975_num1").bounds=(0,0)
    enz_model.reactions.get_by_id("R1975_num2").bounds=(0,0)
    enz_model.reactions.get_by_id("R1975_num3").bounds=(0,0)
    enz_model.reactions.get_by_id("R1975_num4").bounds=(0,0)
    enz_model.reactions.get_by_id("R1975_num5").bounds=(0,0)
    enz_model.reactions.get_by_id(substrate).bounds=(0,3.0676)
    enz_model.reactions.get_by_id('R2399').bounds=(0,0)
    enz_model.reactions.get_by_id('Biomass').bounds=(0.2573,0.2573)
    enz_model_solution = cobra.flux_analysis.pfba(enz_model)
    reac=dict(enz_model_solution.fluxes[abs(enz_model_solution.fluxes) > 1e-10])
    reactionlist=[]
    for i in reac:
        reID=str(enz_model.reactions.get_by_id(i))
        reID=str(enz_model.reactions.get_by_id(i)).split(':')[0]
        reID= reID.split('_')[0]
        reactionlist.append(reID)
    model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    for index in reactionlist:
        for i in enz_model.reactions:
            if '_num'in i.id and index in i.id:
                if float(model_pfba_solution.fluxes[i.id])==0:
                    enz_model.reactions.get_by_id(i.id).bounds=(0,0)
    enz_model.reactions.get_by_id('Biomass').bounds=(0.02573,0.02573)#
    fluxes_outfile = './analysis/eciDL1450_'+name+'_biomass10%'+'.csv'
    enz_model.reactions.get_by_id('Biomass').bounds=(0.02573,0.02573)
    enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,json_model_file)

0.128
Enzyme cost total is:0.04925362433302409
0.128
Enzyme cost total is:0.06041065892102907
0.128
Enzyme cost total is:0.017582311999569143
0.128
Enzyme cost total is:0.06172841540288932


### Biomass yield target of 100%

In [4]:
name_lst=['mal__L','fum','etoh']
for name in name_lst:
    enz_model=get_enzyme_constraint_model(json_model_file)
    product = enz_model.metabolites.get_by_id(name+'_e')
    demand = enz_model.add_boundary(product, type='demand')  # add demand reaction as the objective
    enz_model.objective = demand
    substrate='R2399_reverse'
    enz_model.reactions.get_by_id("R1975_num1").bounds=(0,0)
    enz_model.reactions.get_by_id("R1975_num2").bounds=(0,0)
    enz_model.reactions.get_by_id("R1975_num3").bounds=(0,0)
    enz_model.reactions.get_by_id("R1975_num4").bounds=(0,0)
    enz_model.reactions.get_by_id("R1975_num5").bounds=(0,0)
    enz_model.reactions.get_by_id(substrate).bounds=(0,3.0676)
    enz_model.reactions.get_by_id('Biomass').bounds=(0.2573,0.2573)
    enz_model_solution = cobra.flux_analysis.pfba(enz_model)
    reac=dict(enz_model_solution.fluxes[abs(enz_model_solution.fluxes) > 1e-10])
    reactionlist=[]
    for i in reac:
        reID=str(enz_model.reactions.get_by_id(i))
        reID=str(enz_model.reactions.get_by_id(i)).split(':')[0]
        reID= reID.split('_')[0]
        reactionlist.append(reID)
    model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
        
    for index in reactionlist:
        for i in enz_model.reactions:
            if '_num'in i.id and index in i.id:
                if float(model_pfba_solution.fluxes[i.id])==0:
                    enz_model.reactions.get_by_id(i.id).bounds=(0,0)

    fluxes_outfile = './analysis/eciDL1450_'+name+'biomass100%'+'.csv'

    enz_model_pfba_solution = cobra.flux_analysis.pfba(enz_model)
    enz_model_pfba_solution = get_fluxes_detail_in_model(enz_model,enz_model_pfba_solution,fluxes_outfile,json_model_file)

0.128
Enzyme cost total is:0.12800000000000017
0.128
Enzyme cost total is:0.12800000000000006
0.128
Enzyme cost total is:0.12800000000000014
0.128
Enzyme cost total is:0.12800000000000003


## Introduction of heterogenous pathways

# Top15 of enzyme cost


In [5]:
# malic acid
enzyme_usage1=pd.read_csv('./analysis/eciDL1450_mal__L_biomass10%.csv')
enzyme_usage1=enzyme_usage1.sort_values(['E'],ascending=False).reset_index(drop=True)
enzyme_usage1.head(15)

,Unnamed: 0,fluxes,equ,ec-code,kcat_MW,E
0,R107,2.761032,atp_c + co2_c + h2o_c + pyr_c --> adp_c + 2.0 ...,6.4.1.1,493.871800,0.005591
1,R105_reverse,3.918197,h_c + nadh_c + oaa_c --> mal__L_c + nad_c,NaN,932.974738,0.004200
2,R1963,2.943801,h_c + nadph_c + o2_c --> h2o2_c + nadp_c,NaN,789.385684,0.003729
3,R351,1.001322,e4p_c + xu5p__D_c --> f6p_c + g3p_c,2.2.1.1,317.127316,0.003157
4,R40,4.996164,2pg_c --> h2o_c + pep_c,4.2.1.11,1742.654517,0.002867
5,R61,1.008048,R5P_c + xu5p__D_c --> g3p_c + s7p_c,"2.2.1.1,2.2.1.3",390.600732,0.002581
6,R2158,0.397717,4.0 He_m + 2.0 ficytC_m + qh2_m --> 2.0 FERO_m...,NaN,169.705002,0.002344
7,R2074_num4,3.447814,h2o2_c + meoh_c --> fald_c + 2.0 h2o_c,1.11.1.6,1883.874388,0.001830
8,R2250_num6,3.067600,glc__D_e --> glc__D_c,NaN,1939.388514,0.001582
9,R46,0.526618,coa_m + nad_m + pyr_m --> accoa_m + co2_m + h_...,1.2.4.1,334.549623,0.001574


In [3]:
#malic acid

enzyme_usage1=pd.read_csv('./analysis/eciDL1450_mal__L_biomass10%.csv')
enzyme_usage1=enzyme_usage1.sort_values(['E'],ascending=False).reset_index(drop=True)
enzyme_usage2=pd.read_csv('./analysis/eciDL1450_mal__Lbiomass100%.csv')
enzyme_usage2=enzyme_usage2.sort_values(['E'],ascending=False).reset_index(drop=True)
df=pd.DataFrame()

list=[]
for index in range(len(enzyme_usage1)):
    changeradio=enzyme_usage1.loc[index,'E']/float(enzyme_usage2[enzyme_usage2.iloc[:,0]==enzyme_usage1.iloc[index,0]]['E'])
    if changeradio>=1.5:
        enzyme_usage1.loc[index, 'changeradio'] = changeradio
        list.append(enzyme_usage1.loc[index])
df=pd.DataFrame(list)
df.to_csv('./analysis/mal__LEnhance_TurNup.csv')#enhanced target
df1=pd.DataFrame()
list1=[]
for index in range(len(enzyme_usage1)):
    changeradio=enzyme_usage2.loc[index,'E']/float(enzyme_usage1[enzyme_usage1.iloc[:,0]==enzyme_usage2.iloc[index,0]]['E'])
    if changeradio>=1.5:
        enzyme_usage2.loc[index, 'changeradio'] = changeradio
        list1.append(enzyme_usage2.loc[index])
df1=pd.DataFrame(list1)
df1.to_csv('./analysis/mal__LWeakness_TurNup.csv')#weakened target

enzyme_usage1.head(15)

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning:

divide by zero encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning:

divide by zero encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars



,Unnamed: 0,fluxes,equ,ec-code,kcat_MW,E,changeradio
0,R107,2.761032,atp_c + co2_c + h2o_c + pyr_c --> adp_c + 2.0 ...,6.4.1.1,493.871800,0.005591,2.724544
1,R105_reverse,3.918197,h_c + nadh_c + oaa_c --> mal__L_c + nad_c,NaN,932.974738,0.004200,NaN
2,R1963,2.943801,h_c + nadph_c + o2_c --> h2o2_c + nadp_c,NaN,789.385684,0.003729,inf
3,R351,1.001322,e4p_c + xu5p__D_c --> f6p_c + g3p_c,2.2.1.1,317.127316,0.003157,7.488625
4,R40,4.996164,2pg_c --> h2o_c + pep_c,4.2.1.11,1742.654517,0.002867,NaN
5,R61,1.008048,R5P_c + xu5p__D_c --> g3p_c + s7p_c,"2.2.1.1,2.2.1.3",390.600732,0.002581,5.015897
6,R2158,0.397717,4.0 He_m + 2.0 ficytC_m + qh2_m --> 2.0 FERO_m...,NaN,169.705002,0.002344,NaN
7,R2074_num4,3.447814,h2o2_c + meoh_c --> fald_c + 2.0 h2o_c,1.11.1.6,1883.874388,0.001830,inf
8,R2250_num6,3.067600,glc__D_e --> glc__D_c,NaN,1939.388514,0.001582,53.971343
9,R46,0.526618,coa_m + nad_m + pyr_m --> accoa_m + co2_m + h_...,1.2.4.1,334.549623,0.001574,NaN


In [7]:
# fumaric acid
enzyme_usage1=pd.read_csv('./analysis/eciDL1450_fum_biomass10%.csv')

enzyme_usage1=enzyme_usage1.sort_values(['E'],ascending=False).reset_index(drop=True)
enzyme_usage1.head(15)

,Unnamed: 0,fluxes,equ,ec-code,kcat_MW,E
0,R110,3.918197,mal__L_c --> fum_c + h2o_c,4.2.1.2,347.008310,0.011291
1,R107,2.761032,atp_c + co2_c + h2o_c + pyr_c --> adp_c + 2.0 ...,6.4.1.1,493.871800,0.005591
2,R105_reverse,3.918197,h_c + nadh_c + oaa_c --> mal__L_c + nad_c,NaN,932.974738,0.004200
3,R1963,2.943801,h_c + nadph_c + o2_c --> h2o2_c + nadp_c,NaN,789.385684,0.003729
4,R351,1.001322,e4p_c + xu5p__D_c --> f6p_c + g3p_c,2.2.1.1,317.127316,0.003157
5,R40,4.996164,2pg_c --> h2o_c + pep_c,4.2.1.11,1742.654517,0.002867
6,R61,1.008048,R5P_c + xu5p__D_c --> g3p_c + s7p_c,"2.2.1.1,2.2.1.3",390.600732,0.002581
7,R2158,0.397717,4.0 He_m + 2.0 ficytC_m + qh2_m --> 2.0 FERO_m...,NaN,169.705002,0.002344
8,R2269_reverse,4.341355,fum_c --> fum_e,NaN,2313.412042,0.001877
9,R2074_num4,3.447814,h2o2_c + meoh_c --> fald_c + 2.0 h2o_c,1.11.1.6,1883.874388,0.001830


In [3]:
#fumaric acid

enzyme_usage1=pd.read_csv('./analysis/eciDL1450_fum_biomass10%.csv')
enzyme_usage1=enzyme_usage1.sort_values(['E'],ascending=False).reset_index(drop=True)
enzyme_usage2=pd.read_csv('./analysis/eciDL1450_fumbiomass100%.csv')
enzyme_usage2=enzyme_usage2.sort_values(['E'],ascending=False).reset_index(drop=True)
df=pd.DataFrame()

list=[]
for index in range(len(enzyme_usage1)):
    changeradio=enzyme_usage1.loc[index,'E']/float(enzyme_usage2[enzyme_usage2.iloc[:,0]==enzyme_usage1.iloc[index,0]]['E'])
    if changeradio>=1.5:
        enzyme_usage1.loc[index, 'changeradio'] = changeradio
        list.append(enzyme_usage1.loc[index])
df=pd.DataFrame(list)
df.to_csv('./analysis/fumEnhance_TurNup.csv')#enhanced target
df1=pd.DataFrame()
list1=[]
for index in range(len(enzyme_usage1)):
    changeradio=enzyme_usage2.loc[index,'E']/float(enzyme_usage1[enzyme_usage1.iloc[:,0]==enzyme_usage2.iloc[index,0]]['E'])
    if changeradio>=1.5:
        enzyme_usage2.loc[index, 'changeradio'] = changeradio
        list1.append(enzyme_usage2.loc[index])
df1=pd.DataFrame(list1)
df1.to_csv('./analysis/fumWeakness_TurNup.csv')#weakened target
enzyme_usage1.head(15)

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning:

divide by zero encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning:

invalid value encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning:

divide by zero encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning:

invalid value encountered in double_scalars



,Unnamed: 0,fluxes,equ,ec-code,kcat_MW,E,changeradio
0,R1458,0.052133,lac__L_c + o2_c --> ac_c + co2_c + h2o_c,1.13.12.4,1.157031,0.045057,inf
1,R2157,0.005759,4.0 He_m + h_m + nadh_m + q_m --> 4.0 He_i + n...,1.6.5.9,0.177998,0.032353,6.980417
2,R1037_reverse,1.619475,fald_c + h_c + nadh_c --> meoh_c + nad_c,NaN,151.888498,0.010662,inf
3,R107,2.708899,atp_c + co2_c + h2o_c + pyr_c --> adp_c + 2.0 ...,6.4.1.1,461.607379,0.005868,inf
4,R687_reverse,0.468861,h_m + nadph_m + td2coa_m --> nadp_m + tdcoa_m,NaN,95.563765,0.004906,inf
5,R61,1.008048,R5P_c + xu5p__D_c --> g3p_c + s7p_c,"2.2.1.1,2.2.1.3",403.570784,0.002498,7.627789
6,R351,1.001322,e4p_c + xu5p__D_c --> f6p_c + g3p_c,2.2.1.1,403.570784,0.002481,15.429533
7,R2250_num6,3.067600,glc__D_e --> glc__D_c,NaN,1303.888867,0.002353,240.038102
8,R50,4.996164,g3p_c + nad_c + pi_c --> 13dpg_c + h_c + nadh_c,"1.2.1.12,1.2.1.59",2177.313727,0.002295,NaN
9,R1036,1.997356,atp_c + f6p_c --> adp_c + fdp_B_c + h_c,2.7.1.11,1127.881316,0.001771,NaN


In [9]:
# ethanol
enzyme_usage1=pd.read_csv('./analysis/eciDL1450_etoh_biomass10%.csv')
enzyme_usage1=enzyme_usage1.sort_values(['E'],ascending=False).reset_index(drop=True)
enzyme_usage1.head(15)

,Unnamed: 0,fluxes,equ,ec-code,kcat_MW,E
0,R9,5.715059,h_c + pyr_c --> acald_c + co2_c,4.1.1.1,1200.440609,0.004761
1,R40,6.004212,2pg_c --> h2o_c + pep_c,4.2.1.11,1742.654517,0.003445
2,R2251_num6,2.051484,BDGLC_e --> BDGLC_c,NaN,1939.388514,0.001058
3,R107,0.271623,atp_c + co2_c + h2o_c + pyr_c --> adp_c + 2.0 ...,6.4.1.1,493.871800,0.000550
4,R2249_num6,1.010432,glc__aD_e + h_e --> Glc_aD_c + h_c,NaN,1939.388514,0.000521
5,R110_reverse,0.170323,fum_c + h2o_c --> mal__L_c,NaN,336.830555,0.000506
6,R36_num2,2.694409,adp_c + h_c + pep_c --> atp_c + pyr_c,2.7.1.40,6213.023429,0.000434
7,R51,6.004212,13dpg_c + adp_c --> 3pg_c + atp_c,2.7.2.3,14639.821700,0.000410
8,R1044_num1,3.005404,fdp_B_c --> dhap_c + g3p_c,4.1.2.13,7408.739714,0.000406
9,R10_reverse_num8,5.771227,acald_c + h_c + nadh_c --> etoh_c + nad_c,NaN,15075.254510,0.000383


In [4]:
# ethanol
enzyme_usage1=pd.read_csv('./analysis/eciDL1450_etoh_biomass10%.csv')

enzyme_usage2=pd.read_csv('./analysis/eciDL1450_etohbiomass100%.csv')

df=pd.DataFrame()

list=[]
for index in range(len(enzyme_usage1)):
    changeradio=enzyme_usage1.loc[index,'E']/float(enzyme_usage2[enzyme_usage2.iloc[:,0]==enzyme_usage1.iloc[index,0]]['E'])
    if changeradio>=1.5:
        enzyme_usage1.loc[index, 'changeradio'] = changeradio
        list.append(enzyme_usage1.loc[index])
df=pd.DataFrame(list)
df.to_csv('./analysis/etohEnhance_TurNup1_14.csv')#enhanced target
df1=pd.DataFrame()
list1=[]
for index in range(len(enzyme_usage1)):
    changeradio=enzyme_usage2.loc[index,'E']/float(enzyme_usage1[enzyme_usage1.iloc[:,0]==enzyme_usage2.iloc[index,0]]['E'])
    if changeradio>=1.5:
        enzyme_usage1.loc[index, 'changeradio'] = changeradio
        list1.append(enzyme_usage1.loc[index])
df1=pd.DataFrame(list1)
df1.to_csv('./analysis/etohWeakness_TurNup1_14.csv')#weakened target


enzyme_usage1.head(15)

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

invalid value encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning:

divide by zero encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning:

invalid value encountered in double_scalars

E:\Anaconda\envs\cobra\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning:

divide by zero encountered in double_scalars



,Unnamed: 0,fluxes,equ,ec-code,kcat_MW,E,changeradio
0,R1,3.067600,glc__D_e --> 0.67 BDGLC_e + 0.33 glc__aD_e,NaN,NaN,NaN,NaN
1,R2,0.000000,glc__D_c --> 0.67 BDGLC_c + 0.33 Glc_aD_c,NaN,NaN,NaN,NaN
2,R3,0.000000,Glc_aD_c --> BDGLC_c,5.1.3.3,625066.591111,0.000000,NaN
3,R4,0.000000,arab__L_c --> BLARAB_c,5.1.3.3,625066.591111,0.000000,NaN
4,R5,0.000000,fru_B_c --> fru_c,5.1.3.3,625066.591111,0.000000,NaN
5,R6_num1,2.051484,BDGLC_c + atp_c --> adp_c + g6p_B_c + h_c,"2.7.1.1,2.7.1.2",3328.716016,0.000616,NaN
6,R7,2.051484,g6p_B_c --> f6p_c,5.3.1.9,178726.601902,0.000011,NaN
7,R8,0.000000,cit_m --> ac_m + oaa_m,4.1.3.6,NaN,NaN,NaN
8,R9,5.766755,h_c + pyr_c --> acald_c + co2_c,4.1.1.1,2620.070691,0.002201,6.045869
9,R10_num1,0.000000,etoh_c + nad_c --> acald_c + h_c + nadh_c,"1.1.1.1,1.1.1.71",35847.749665,0.000000,NaN
